In [15]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf 
import os
from datetime import datetime
from pathlib import Path
# 

In [16]:
# Setup the Model Parameters
INPUT_SIZE=30
HIDDEN_SIZE=1000
OUTPUT_SIZE = 10  
START_LEARNING_RATE=1e-3
BATCH_LENGTH=25
NUM_TRAINING_STEPS = 100
EPOCHS=15
TEST_LENGTH=125
DISPLAY_STEP=25
weight_learning_rate=1e-3

log_dir = os.environ['HOME']+"/MyData/KeRNL/logs/ffn/lin_func_approx/kernl_eta_weight_%1.0e_batch_%1.0e_hum_hidd_%1.0e_steps_%1.0e_run_%s" %(weight_learning_rate,BATCH_LENGTH,HIDDEN_SIZE,NUM_TRAINING_STEPS, datetime.now().strftime("%Y%m%d_%H%M"))
log_dir

'/home/eghbal/MyData/KeRNL/logs/ffn/lin_func_approx/kernl_eta_weight_1e-03_batch_2e+01_hum_hidd_1e+03_steps_1e+02_run_20190304_1735'

In [17]:
Target=np.random.uniform(low=-1,high=1,size=(30,10))
x_train=np.random.uniform(low=0,high=1,size=(BATCH_LENGTH*NUM_TRAINING_STEPS,30))
y_train=np.matmul(x_train,Target)
#
x_test=np.random.uniform(low=0,high=1,size=(EPOCHS*TEST_LENGTH,30))
y_test=np.matmul(x_test,Target)


In [18]:
tf.reset_default_graph()
graph=tf.Graph()
with graph.as_default():

    #
    BATCH_SIZE=tf.placeholder(tf.int64)
    X = tf.placeholder('float', shape=[None, INPUT_SIZE])  
    Y = tf.placeholder('float', shape=[None, OUTPUT_SIZE])
    # define a dataset
    dataset=tf.data.Dataset.from_tensor_slices((X,Y)).batch(BATCH_SIZE).repeat()
    dataset = dataset.shuffle(buffer_size=500)
    iter = dataset.make_initializable_iterator()
    inputs,labels =iter.get_next()
    
    
    #
    initializer = tf.random_normal_initializer(stddev=0.1)
     # Hidden Layer Variables
    W_1 = tf.get_variable("Hidden_W", shape=[INPUT_SIZE, HIDDEN_SIZE], initializer=initializer)
    b_1 = tf.get_variable("Hidden_b", shape=[HIDDEN_SIZE], initializer=initializer)
  # output layer variables 
    W_2 = tf.get_variable("Output_W", shape=[HIDDEN_SIZE, OUTPUT_SIZE], initializer=initializer)
    b_2 = tf.get_variable("Output_b", shape=[OUTPUT_SIZE], initializer=initializer)
    # return weight 
    B=tf.get_variable('B',shape=[OUTPUT_SIZE,HIDDEN_SIZE],initializer=tf.initializers.random_uniform(minval=-0.5,maxval=0.5))
    trainables=[W_1,b_1,W_2,b_2,B]
    variable_names=[v.name for v in tf.trainable_variables()]
    #
    #define transformation from input to output  
  # Hidden Layer Transformation
    hidden=tf.matmul(inputs, W_1) + b_1
  # Output Layer Transformation
    output = tf.matmul(hidden, W_2) + b_2
    ##################
    ## kernl train ####
    ##################
    with tf.name_scope("kernl_train") as scope:
        loss = tf.losses.mean_squared_error(labels, output)
        nse=tf.reduce_mean(tf.square(tf.subtract(output,tf.expand_dims(tf.reduce_mean(output,axis=0),axis=0))))
        correct_prediction = tf.equal(tf.argmax(labels, 1), tf.argmax(output, 1))
        accuracy = 100 * tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        optimizer  = tf.train.AdamOptimizer(learning_rate=weight_learning_rate)
        
  # compute and apply gradiants
        dW_2=tf.reduce_mean(tf.transpose(tf.einsum('uv,un->uvn',tf.subtract(output,labels),(hidden))),axis=-1)
        db_2=tf.reduce_mean(tf.subtract(output,labels),axis=0)
        dW_1=tf.transpose(tf.reduce_mean(tf.einsum('uv,ug->uvg',tf.einsum('un,nv->uv',tf.subtract(output,labels),B),inputs),axis=0))
        db_1=tf.transpose(tf.reduce_mean(tf.einsum('uv,ug->ug',tf.einsum('un,nv->uv',tf.subtract(output,labels),B),hidden*0+1),axis=0)) 
        # gradient for B
        dB=tf.negative(tf.reduce_mean(tf.einsum('uv,uz->uvz',output,tf.subtract(hidden,tf.einsum('uv,vz->uz',output,B))),axis=0))
        new_ffn_grads=list(zip([dW_1,db_1,dW_2,db_2,dB],trainables))
        ffn_train_op=optimizer.apply_gradients(new_ffn_grads)
        
  
    with tf.name_scope("evaluate") as scope: 
        kernl_loss_cross_validiation=tf.losses.mean_squared_error(labels,output)
        kernl_correct_pred_cross_val=tf.equal(tf.argmax(labels, 1), tf.argmax(output, 1))
        kernl_accu_cross_validation=100 * tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        
    with tf.name_scope('cross_validation_summary') as scope: 
        tf.summary.scalar('cross_validation_loss',kernl_loss_cross_validiation+1e-10)
        tf.summary.scalar('cross_validation_accu',kernl_accu_cross_validation+1e-10)
      
        kernl_evaluate_summary_op=tf.summary.merge_all(scope="cross_validation_summary") 
        
    ##################
    # SUMMARIES ######
    ##################
                
    with tf.name_scope("summaries") as scope:
                    # kernl kernel
        tf.summary.histogram('kernl_hidd_W',W_1+1e-10)
        tf.summary.histogram('return_B',B+1e-10)
                    # kernl output weight
        tf.summary.histogram('kernl_output_W',W_2+1e-10)
                    # kernl output bias
                    # kernl loss and accuracy
        tf.summary.scalar('loss_output_prediction',loss+1e-10)
        tf.summary.scalar('accuracy',accuracy+1e-10)
        tf.summary.scalar('NSE',nse+1e-10)
        kernl_merged_summary_op=tf.summary.merge_all(scope="summaries")          
        
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()

In [19]:
# verify initialization
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

with tf.Session(graph=graph,) as sess : 
    sess.run(init)
    values,trainable_vars = sess.run([variable_names,trainables])
    for k, v in zip(variable_names,values):
        print(["variable: " , k])
        #print(["value: " , v])
        print(["variable: " , np.unicode_.find(k,'output')]) 
        print(["shape: " , v.shape])
        #print(v) training_steps

['variable: ', 'Hidden_W:0']
['variable: ', -1]
['shape: ', (30, 1000)]
['variable: ', 'Hidden_b:0']
['variable: ', -1]
['shape: ', (1000,)]
['variable: ', 'Output_W:0']
['variable: ', -1]
['shape: ', (1000, 10)]
['variable: ', 'Output_b:0']
['variable: ', -1]
['shape: ', (10,)]
['variable: ', 'B:0']
['variable: ', -1]
['shape: ', (10, 1000)]


In [20]:
Path(log_dir).mkdir(exist_ok=True, parents=True)
filelist = [ f for f in os.listdir(log_dir) if f.endswith(".local") ]
for f in filelist:
    os.remove(os.path.join(log_dir, f))

In [21]:
# write graph into tensorboard 
tb_writer = tf.summary.FileWriter(log_dir,graph)
# run a training session 
with tf.Session(graph=graph) as sess:
    sess.run(init)
    for epoch in range(EPOCHS):
        sess.run(iter.initializer,feed_dict={X: x_train, Y: y_train , BATCH_SIZE: BATCH_LENGTH})
        for step in range(NUM_TRAINING_STEPS): 
            _, kernl_loss,kernl_accu,kernl_merged_summary=sess.run([ffn_train_op,loss,accuracy,kernl_merged_summary_op])
            tb_writer.add_summary(kernl_merged_summary, global_step=epoch*NUM_TRAINING_STEPS+step+1)
            if step % DISPLAY_STEP==0 : 
                print('Epoch: {}, Batch: {} , total batch {}, total trials: {},kernl train Loss: {:.3f}, kernl_accuracy : {:.3f}'.format(epoch+1,step + 1,epoch*NUM_TRAINING_STEPS+step+1,(epoch*NUM_TRAINING_STEPS+step+1)*BATCH_LENGTH, kernl_loss,kernl_accu))
        sess.run(iter.initializer, feed_dict={ X: x_test, Y: y_test, BATCH_SIZE: TEST_LENGTH})        
        # run test at the end of each epoch  
        kernl_test_loss,kernl_test_accu, kernl_evaluate_summary=sess.run([kernl_loss_cross_validiation,kernl_accu_cross_validation,kernl_evaluate_summary_op])        
        tb_writer.add_summary(kernl_evaluate_summary, global_step=epoch*NUM_TRAINING_STEPS+NUM_TRAINING_STEPS+1)
        print('Epoch: {}, cross validation loss :{:.3f}, cross validation accuracy: {:.3f}'.format(epoch+1,kernl_test_loss,kernl_test_accu))
    print("Optimization Finished!")
    save_path = saver.save(sess, log_dir+"/model.ckpt", global_step=step,write_meta_graph=True)
    print("Model saved in path: %s" % save_path)


Epoch: 1, Batch: 1 , total batch 1, total trials: 25,kernl train Loss: 5.789, kernl_accuracy : 0.000
Epoch: 1, Batch: 26 , total batch 26, total trials: 650,kernl train Loss: 0.905, kernl_accuracy : 84.000
Epoch: 1, Batch: 51 , total batch 51, total trials: 1275,kernl train Loss: 0.767, kernl_accuracy : 88.000
Epoch: 1, Batch: 76 , total batch 76, total trials: 1900,kernl train Loss: 0.433, kernl_accuracy : 72.000
Epoch: 1, cross validation loss :0.284, cross validation accuracy: 80.800
Epoch: 2, Batch: 1 , total batch 101, total trials: 2525,kernl train Loss: 0.314, kernl_accuracy : 64.000
Epoch: 2, Batch: 26 , total batch 126, total trials: 3150,kernl train Loss: 0.212, kernl_accuracy : 76.000
Epoch: 2, Batch: 51 , total batch 151, total trials: 3775,kernl train Loss: 0.109, kernl_accuracy : 92.000
Epoch: 2, Batch: 76 , total batch 176, total trials: 4400,kernl train Loss: 0.055, kernl_accuracy : 84.000
Epoch: 2, cross validation loss :0.041, cross validation accuracy: 88.800
Epoch: 